In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [3]:

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [4]:
from itertools import combinations

In [5]:
import seaborn as sns
sns.set_style("whitegrid")

In [6]:
state = 'GA'

In [7]:
file_symptoms_train = '../comments/raw_comments/agent_comments_'+state+\
                        '_filtered_final_joined_selected_columns_use_max_scale_one_df_train.fea'

noun_sentence_state_train = pd.read_feather(file_symptoms_train)

In [8]:
file_symptoms_test = '../comments/raw_comments/agent_comments_'+state+\
                        '_filtered_final_joined_selected_columns_use_max_scale_one_df_test.fea'
noun_sentence_state_test = pd.read_feather(file_symptoms_test)


In [9]:
file = 'NEW_AVM_DATA'+state+'.fea'

new_avm_df = pd.read_feather(file)
new_avm_df = new_avm_df.drop(columns=['index','state'])

In [10]:
new_avm_df = new_avm_df.rename(columns={'avmvalue':'new_avmValue', 'avmerror':'new_avmerror'})

In [11]:
noun_sentence_state_train = noun_sentence_state_train.merge(new_avm_df,
                                                         how='left', left_on='listingid', right_on='listingid')

In [12]:
noun_sentence_state_train = noun_sentence_state_train.dropna()

In [13]:
noun_sentence_state_test = noun_sentence_state_test.merge(new_avm_df,
                                                         how='left', left_on='listingid', right_on='listingid')

In [14]:
noun_sentence_state_test = noun_sentence_state_test.dropna()

In [15]:
noun_sentence_state_test_transdate_train = noun_sentence_state_train.groupby('Transdate').mean()[['new_avmerror']].reset_index()
noun_sentence_state_test_transdate_train = noun_sentence_state_test_transdate_train.rename(columns={"new_avmerror": "avmerror_mean"})

In [16]:
noun_sentence_state_test_transdate_test = noun_sentence_state_test.groupby('Transdate').mean()[['new_avmerror']].reset_index()
noun_sentence_state_test_transdate_test = noun_sentence_state_test_transdate_test.rename(columns={"new_avmerror": "avmerror_mean"})

In [17]:
noun_sentence_state_train = noun_sentence_state_train.merge(noun_sentence_state_test_transdate_train,
                                                         how='left', left_on='Transdate', right_on='Transdate')

In [18]:
noun_sentence_state_test = noun_sentence_state_test.merge(noun_sentence_state_test_transdate_test,
                                                         how='left', left_on='Transdate', right_on='Transdate')

In [19]:
noun_sentence_state_test_transdate_test

,Transdate,avmerror_mean
0,20180102.0,0.047311
1,20180103.0,-0.018236
2,20180104.0,-0.028947
3,20180105.0,-0.014688
4,20180106.0,-0.304093
...,...,...
987,20210624.0,0.057263
988,20210625.0,0.059909
989,20210628.0,0.081728
990,20210629.0,0.047540


In [20]:
#selcted_columns = selcted_columns + list(noun_sentence_state_train.columns[51:-1])

In [21]:
all_combinations = [['new_avmValue','cbsa'],
                    ['new_avmValue','cbsa', 'cj_living_area'],
                    ['new_avmValue','cbsa', 'cj_living_area', 'census_tract'],
                    ['new_avmValue','cbsa', 'cj_living_area', 'census_tract', 'Transdate'],
                    ['new_avmValue','cbsa', 'cj_living_area', 'census_tract',
                   'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1',
                   'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3',
                   'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5',
                   'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7',
                   'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9',
                   'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11',
                   'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse',
                   'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1',
                   'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3',
                   'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5',
                   'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7',
                   'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9',
                   'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11',
                   'AVM_Error_Indicators_0_max_value',
                   'AVM_Error_Indicators_0_min_value',
                   'AVM_Error_Indicators_0_max_indicator',
                   'AVM_Error_Indicators_0_min_indicator',
                   'AVM_Error_Indicators_0_diff_indicator',
                   'AVM_Error_Indicators_0_diff_value', 'AVM_Error_Indicators_0_mean','avmerror_mean'] +\
                    list(noun_sentence_state_train.columns[51:-3]),
                   ['new_avmValue','cbsa', 'cj_living_area',
                   'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1',
                   'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3',
                   'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5',
                   'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7',
                   'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9',
                   'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11',
                   'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse',
                   'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1',
                   'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3',
                   'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5',
                   'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7',
                   'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9',
                   'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11',
                   'AVM_Error_Indicators_0_max_value',
                   'AVM_Error_Indicators_0_min_value',
                   'AVM_Error_Indicators_0_max_indicator',
                   'AVM_Error_Indicators_0_min_indicator',
                   'AVM_Error_Indicators_0_diff_indicator',
                   'AVM_Error_Indicators_0_diff_value', 'AVM_Error_Indicators_0_mean','avmerror_mean'] +\
                    list(noun_sentence_state_train.columns[51:-3]),
                   ['new_avmValue','cbsa', 'cj_living_area', 'Transdate', 
                   'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1',
                   'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3',
                   'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5',
                   'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7',
                   'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9',
                   'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11',
                   'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse',
                   'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1',
                   'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3',
                   'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5',
                   'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7',
                   'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9',
                   'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11',
                   'AVM_Error_Indicators_0_max_value',
                   'AVM_Error_Indicators_0_min_value',
                   'AVM_Error_Indicators_0_max_indicator',
                   'AVM_Error_Indicators_0_min_indicator',
                   'AVM_Error_Indicators_0_diff_indicator',
                   'AVM_Error_Indicators_0_diff_value', 'AVM_Error_Indicators_0_mean','avmerror_mean'] +\
                    list(noun_sentence_state_train.columns[51:-3]),
                   ['new_avmValue','cbsa', 'cj_living_area', 'census_tract', 'Transdate', 
                   'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1',
                   'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3',
                   'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5',
                   'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7',
                   'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9',
                   'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11',
                   'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse',
                   'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1',
                   'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3',
                   'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5',
                   'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7',
                   'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9',
                   'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11',
                   'AVM_Error_Indicators_0_max_value',
                   'AVM_Error_Indicators_0_min_value',
                   'AVM_Error_Indicators_0_max_indicator',
                   'AVM_Error_Indicators_0_min_indicator',
                   'AVM_Error_Indicators_0_diff_indicator',
                   'AVM_Error_Indicators_0_diff_value', 'AVM_Error_Indicators_0_mean','avmerror_mean'] +\
                    list(noun_sentence_state_train.columns[51:-3])]

In [22]:
list(noun_sentence_state_train.columns[51:-3])[-1]

'new granite counter tops, new hardwood floors, new stainless steel appliances'

In [23]:
all_combinations_names = ['new_avmValue-cbsa', 
                          'new_avmValue-cbsa-cj_living_area',
                          'new_avmValue-cbsa-cj_living_area-census_tract',
                          'new_avmValue-cbsa-cj_living_area-census_tract-Transdate',
                          'new_avmValue-cbsa-cj_living_area-census_tract-comments',
                          'new_avmValue-cbsa-cj_living_area-comments',
                          'new_avmValue-cbsa-cj_living_area-comments-Transdate',
                          'new_avmValue-cbsa-cj_living_area-comments-census_tract-Transdate']

In [24]:
noun_sentence_state_test['Train/Test'] = 'Test'
noun_sentence_state_train['Train/Test'] = 'Train'

In [25]:
for names in all_combinations_names:
    noun_sentence_state_test[names] = -9999
    noun_sentence_state_train[names] = -9999

In [26]:
noun_sentence_state_test

,listingid,qtr,cbsa_div,cbsa,state,Transdate,Transprice,cj_living_area,fips_code,census_tract,...,avmerror_mean,Train/Test,new_avmValue-cbsa,new_avmValue-cbsa-cj_living_area,new_avmValue-cbsa-cj_living_area-census_tract,new_avmValue-cbsa-cj_living_area-census_tract-Transdate,new_avmValue-cbsa-cj_living_area-census_tract-comments,new_avmValue-cbsa-cj_living_area-comments,new_avmValue-cbsa-cj_living_area-comments-Transdate,new_avmValue-cbsa-cj_living_area-comments-census_tract-Transdate
0,1066,201802.0,23580,23580,GA,20180111.0,150000.0,925.0,13139,13139001608,...,-0.016037,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
1,1097,201802.0,12060,12060,GA,20180221.0,300000.0,3217.0,13151,13151070203,...,-0.076224,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
2,1191,201802.0,12060,12060,GA,20180328.0,377500.0,3061.0,13117,13117130610,...,0.022623,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
3,1193,201802.0,12060,12060,GA,20180122.0,210000.0,2765.0,13135,13135050728,...,-0.025617,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
4,3926,201802.0,12060,12060,GA,20180328.0,533000.0,4310.0,13067,13067030335,...,0.022623,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56703,251039161,202105.0,12060,12060,GA,20210625.0,371200.0,3223.0,13113,13113140303,...,0.059909,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
56704,251054943,202105.0,12060,12060,GA,20210615.0,195000.0,2129.0,13063,13063040513,...,0.023533,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
56705,251227415,202105.0,12060,12060,GA,20210629.0,150000.0,2157.0,13135,13135050436,...,0.047540,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
56706,251442877,202105.0,12060,12060,GA,20210624.0,280000.0,2973.0,13121,13121010514,...,0.057263,Test,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999


In [27]:
import joblib

In [28]:
for idd,list_preds in enumerate(all_combinations):
    print(list(list_preds))
    
    selcted_columns = list_preds
    symptoms_df = noun_sentence_state_train.loc[:,selcted_columns]
    symptoms_df_test = noun_sentence_state_test.loc[:,selcted_columns]
    
    Predictors = symptoms_df.to_numpy()
    X_test = symptoms_df_test.to_numpy()
    X_train = Predictors

    y_train_regressor = noun_sentence_state_train.new_avmerror.values
    y_test_regressor = noun_sentence_state_test.new_avmerror.values
    
    filename = 'LGBM_regressor_additive_'+state+'_'+str(idd)+'_new_avm.sav'
    model_regressor = joblib.load(filename)
    
    predictions_train_Regressor = model_regressor.predict(X_train)
    predictions_test_Regressor = model_regressor.predict(X_test)
    
    avmValue_test = noun_sentence_state_test['new_avmValue']
    avmValue_train = noun_sentence_state_train['new_avmValue']

    transprice_test = noun_sentence_state_test['Transprice']
    transprice_train = noun_sentence_state_train['Transprice']

    predictions_test_valueR = avmValue_test/(1-predictions_test_Regressor)
    predictions_train_valueR = avmValue_train/(1-predictions_train_Regressor)
    
    
    print('') 
    print('Testing')
    print(np.mean(np.abs(transprice_test-predictions_test_valueR)/\
              (transprice_test)))
    print(np.mean(np.abs(transprice_test - avmValue_test)/\
                  (transprice_test)))

    print('') 
    print('Training') 
    print(np.mean(np.abs(transprice_train-predictions_train_valueR)/\
              (transprice_train)))
    print(np.mean(np.abs(transprice_train - avmValue_train)/\
                  (transprice_train)))   
    
    print(' ')
    
    noun_sentence_state_test[all_combinations_names[idd]] = predictions_test_valueR
    noun_sentence_state_train[all_combinations_names[idd]] = predictions_train_valueR

['new_avmValue', 'cbsa']

Testing
0.11030085367601969
0.11038717600929503

Training
0.11027277367948198
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area']

Testing
0.11062738237396857
0.11038717600929503

Training
0.10819057621550902
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area', 'census_tract']

Testing
0.1112341799725681
0.11038717600929503

Training
0.10628130926620227
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area', 'census_tract', 'Transdate']

Testing
0.10858128526617714
0.11038717600929503

Training
0.09968291961707769
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area', 'census_tract', 'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1', 'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3', 'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5', 'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7', 'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9', 'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11', 'AVM_Error_Indicators_sca


Testing
0.09903124073050479
0.11038717600929503

Training
0.08578432068086382
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area', 'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1', 'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3', 'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5', 'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7', 'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9', 'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11', 'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse', 'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1', 'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3', 'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5', 'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7', 'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9', 'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11', 'AVM_Error_Indicators_0_max_value', 'AVM_Error_Indicators_0_min_value', 'AVM_Error_Indicators_0_max_indicator', 


Testing
0.10062278746362417
0.11038717600929503

Training
0.08928870586666764
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area', 'Transdate', 'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1', 'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3', 'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5', 'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7', 'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9', 'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11', 'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse', 'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1', 'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3', 'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5', 'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7', 'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9', 'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11', 'AVM_Error_Indicators_0_max_value', 'AVM_Error_Indicators_0_min_value', 'AVM_Error_Indicators_0_max


Testing
0.10005957328363206
0.11038717600929503

Training
0.08815258694061792
0.11059679076013033
 
['new_avmValue', 'cbsa', 'cj_living_area', 'census_tract', 'Transdate', 'AVM_Error_Indicators_0', 'AVM_Error_Indicators_1', 'AVM_Error_Indicators_2', 'AVM_Error_Indicators_3', 'AVM_Error_Indicators_4', 'AVM_Error_Indicators_5', 'AVM_Error_Indicators_6', 'AVM_Error_Indicators_7', 'AVM_Error_Indicators_8', 'AVM_Error_Indicators_9', 'AVM_Error_Indicators_10', 'AVM_Error_Indicators_11', 'AVM_Error_Indicators_scale', 'AVM_Error_Indicators_scale_inverse', 'Listing_Error_Indicators_0', 'Listing_Error_Indicators_1', 'Listing_Error_Indicators_2', 'Listing_Error_Indicators_3', 'Listing_Error_Indicators_4', 'Listing_Error_Indicators_5', 'Listing_Error_Indicators_6', 'Listing_Error_Indicators_7', 'Listing_Error_Indicators_8', 'Listing_Error_Indicators_9', 'Listing_Error_Indicators_10', 'Listing_Error_Indicators_11', 'AVM_Error_Indicators_0_max_value', 'AVM_Error_Indicators_0_min_value', 'AVM_Error_


Testing
0.09811827755948811
0.11038717600929503

Training
0.08390867180763845
0.11059679076013033
 


## DATA FOR THU

In [29]:
noun_sentence_state_full = noun_sentence_state_train.copy(deep=True)

In [30]:
noun_sentence_state_full = noun_sentence_state_full.append(noun_sentence_state_test)

In [31]:
len(noun_sentence_state_full)

189177

In [32]:
noun_sentence_state_full = noun_sentence_state_full.sort_values('Transdate')

In [33]:
noun_sentence_state_full.columns[:17]

Index(['listingid', 'qtr', 'cbsa_div', 'cbsa', 'state', 'Transdate',
       'Transprice', 'cj_living_area', 'fips_code', 'census_tract', 'zip',
       'ListingPrice', 'avmValue', 'avmerror', 'AVM_Error_List',
       'Listing_Error_List', 'AVM_Error_Indicators_0'],
      dtype='object')

In [34]:
all_combinations_names

['new_avmValue-cbsa',
 'new_avmValue-cbsa-cj_living_area',
 'new_avmValue-cbsa-cj_living_area-census_tract',
 'new_avmValue-cbsa-cj_living_area-census_tract-Transdate',
 'new_avmValue-cbsa-cj_living_area-census_tract-comments',
 'new_avmValue-cbsa-cj_living_area-comments',
 'new_avmValue-cbsa-cj_living_area-comments-Transdate',
 'new_avmValue-cbsa-cj_living_area-comments-census_tract-Transdate']

In [35]:
list_of_colums = ['listingid', 'qtr', 'state', 'cbsa',  'cj_living_area', 
                  'census_tract', 'Transdate', 'Transprice', 
                   'ListingPrice', 'new_avmValue', 'new_avmerror', 'Train/Test'] + all_combinations_names

In [36]:
list_of_colums

['listingid',
 'qtr',
 'state',
 'cbsa',
 'cj_living_area',
 'census_tract',
 'Transdate',
 'Transprice',
 'ListingPrice',
 'new_avmValue',
 'new_avmerror',
 'Train/Test',
 'new_avmValue-cbsa',
 'new_avmValue-cbsa-cj_living_area',
 'new_avmValue-cbsa-cj_living_area-census_tract',
 'new_avmValue-cbsa-cj_living_area-census_tract-Transdate',
 'new_avmValue-cbsa-cj_living_area-census_tract-comments',
 'new_avmValue-cbsa-cj_living_area-comments',
 'new_avmValue-cbsa-cj_living_area-comments-Transdate',
 'new_avmValue-cbsa-cj_living_area-comments-census_tract-Transdate']

In [37]:
df_Thu = noun_sentence_state_full.loc[:,list_of_colums]

In [38]:
file_symptoms = 'agent_comments_'+state+\
                        '_for_Thu_01_new.fea'
df_Thu.reset_index().to_feather(file_symptoms)

In [39]:
file_symptoms = 'agent_comments_'+state+\
                        '_for_Thu_01_new.csv'
df_Thu.reset_index().to_csv(file_symptoms)

In [40]:
df_Thu.columns

Index(['listingid', 'qtr', 'state', 'cbsa', 'cj_living_area', 'census_tract',
       'Transdate', 'Transprice', 'ListingPrice', 'new_avmValue',
       'new_avmerror', 'Train/Test', 'new_avmValue-cbsa',
       'new_avmValue-cbsa-cj_living_area',
       'new_avmValue-cbsa-cj_living_area-census_tract',
       'new_avmValue-cbsa-cj_living_area-census_tract-Transdate',
       'new_avmValue-cbsa-cj_living_area-census_tract-comments',
       'new_avmValue-cbsa-cj_living_area-comments',
       'new_avmValue-cbsa-cj_living_area-comments-Transdate',
       'new_avmValue-cbsa-cj_living_area-comments-census_tract-Transdate'],
      dtype='object')